In [61]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import sqlite3

# loading data from python.db database
con = sqlite3.connect('python.db')

# import 'school' and 'student' tables into pandas dataframe
school_data = pd.read_sql_query('SELECT * FROM school', con)
student_data = pd.read_sql_query('SELECT * FROM student', con)

In [63]:
# join the two tables into a single dataframe
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,id_x,Student ID,student_name,gender,grade,school_name,reading_score,math_score,id_y,School ID,type,size,budget
0,0,0,Paul Bradley,M,9th,Huang High School,66,79,0,0,District,2917,1910635
1,1,1,Victor Smith,M,12th,Huang High School,94,61,0,0,District,2917,1910635
2,2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,0,District,2917,1910635
3,3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,0,District,2917,1910635
4,4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,0,District,2917,1910635


In [89]:
total_schools = school_data['school_name'].nunique()

In [91]:
total_students = student_data['Student ID'].nunique()

In [93]:
total_budget = school_data['budget'].sum()

In [95]:
average_math_score = student_data['math_score'].mean()

In [79]:
avg_reading_score = school_data_complete['reading_score'].mean()

In [97]:
average_reading_score = student_data['reading_score'].mean()

In [99]:
overall_passing_rate = (average_math_score + average_reading_score) / 2

In [101]:
passing_math_count = student_data[student_data['math_score'] >= 70].shape[0]
percentage_passing_math = (passing_math_count / total_students) * 100

In [103]:
passing_reading_count = student_data[student_data['reading_score'] >= 70].shape[0]
percentage_passing_reading = (passing_reading_count / total_students) * 100

In [107]:
district_summary = pd.DataFrame({
    "Total Schools": [total_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math_score],
    "Average Reading Score": [average_reading_score],
    "Overall Passing Rate": [overall_passing_rate],
    "Percentage Passing Math": [percentage_passing_math],
    "Percentage Passing Reading": [percentage_passing_reading]
})

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Overall Passing Rate,Percentage Passing Math,Percentage Passing Reading
0,15,39170,24649428,78.985371,81.87784,80.431606,74.980853,85.805463


In [109]:
school_summary = school_data_complete.groupby('school_name').agg(
    School_Type=('type', 'first'),
    Total_Students=('Student ID', 'count'),
    Total_Budget=('budget', 'first'),
    Average_Math_Score=('math_score', 'mean'),
    Average_Reading_Score=('reading_score', 'mean')
).reset_index()

In [113]:
school_summary['Per_Student_Budget'] = school_summary['Total_Budget'] / school_summary['Total_Students']

In [115]:
school_summary['% Passing Math'] = (school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name').size() / school_summary['Total_Students']) * 100
school_summary['% Passing Reading'] = (school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name').size() / school_summary['Total_Students']) * 100
school_summary['Overall Passing Rate'] = (school_summary['% Passing Math'] + school_summary['% Passing Reading']) / 2

In [117]:
school_summary

,school_name,School_Type,Total_Students,Total_Budget,Average_Math_Score,Average_Reading_Score,Per_Student_Budget,% Passing Math,% Passing Reading,Overall Passing Rate
0,Bailey High School,District,4976,3124928,77.048432,81.033963,628.0,NaN,NaN,NaN
1,Cabrera High School,Charter,1858,1081356,83.061895,83.975780,582.0,NaN,NaN,NaN
2,Figueroa High School,District,2949,1884411,76.711767,81.158020,639.0,NaN,NaN,NaN
3,Ford High School,District,2739,1763916,77.102592,80.746258,644.0,NaN,NaN,NaN
4,Griffin High School,Charter,1468,917500,83.351499,83.816757,625.0,NaN,NaN,NaN
5,Hernandez High School,District,4635,3022020,77.289752,80.934412,652.0,NaN,NaN,NaN
6,Holden High School,Charter,427,248087,83.803279,83.814988,581.0,NaN,NaN,NaN
7,Huang High School,District,2917,1910635,76.629414,81.182722,655.0,NaN,NaN,NaN
8,Johnson High School,District,4761,3094650,77.072464,80.966394,650.0,NaN,NaN,NaN
9,Pena High School,Charter,962,585858,83.839917,84.044699,609.0,NaN,NaN,NaN


In [119]:
top_performing_schools = school_summary.sort_values(by='Overall Passing Rate', ascending=False).head(5)
top_performing_schools

,school_name,School_Type,Total_Students,Total_Budget,Average_Math_Score,Average_Reading_Score,Per_Student_Budget,% Passing Math,% Passing Reading,Overall Passing Rate
0,Bailey High School,District,4976,3124928,77.048432,81.033963,628.0,NaN,NaN,NaN
1,Cabrera High School,Charter,1858,1081356,83.061895,83.975780,582.0,NaN,NaN,NaN
2,Figueroa High School,District,2949,1884411,76.711767,81.158020,639.0,NaN,NaN,NaN
3,Ford High School,District,2739,1763916,77.102592,80.746258,644.0,NaN,NaN,NaN
4,Griffin High School,Charter,1468,917500,83.351499,83.816757,625.0,NaN,NaN,NaN


In [125]:
bottom_performing_schools = school_summary.sort_values(by='Overall Passing Rate').head(5)
bottom_performing_schools

,school_name,School_Type,Total_Students,Total_Budget,Average_Math_Score,Average_Reading_Score,Per_Student_Budget,% Passing Math,% Passing Reading,Overall Passing Rate
0,Bailey High School,District,4976,3124928,77.048432,81.033963,628.0,NaN,NaN,NaN
1,Cabrera High School,Charter,1858,1081356,83.061895,83.975780,582.0,NaN,NaN,NaN
2,Figueroa High School,District,2949,1884411,76.711767,81.158020,639.0,NaN,NaN,NaN
3,Ford High School,District,2739,1763916,77.102592,80.746258,644.0,NaN,NaN,NaN
4,Griffin High School,Charter,1468,917500,83.351499,83.816757,625.0,NaN,NaN,NaN


In [129]:
average_math_by_grade = school_data_complete.groupby(['school_name', 'grade'])['math_score'].mean().unstack()
average_reading_by_grade = school_data_complete.groupby(['school_name', 'grade'])['reading_score'].mean().unstack()

average_math_by_grade, average_reading_by_grade

(grade                       10th       11th       12th        9th
 school_name                                                      
 Bailey High School     76.996772  77.515588  76.492218  77.083676
 Cabrera High School    83.154506  82.765560  83.277487  83.094697
 Figueroa High School   76.539974  76.884344  77.151369  76.403037
 Ford High School       77.672316  76.918058  76.179963  77.361345
 Griffin High School    84.229064  83.842105  83.356164  82.044010
 Hernandez High School  77.337408  77.136029  77.186567  77.438495
 Holden High School     83.429825  85.000000  82.855422  83.787402
 Huang High School      75.908735  76.446602  77.225641  77.027251
 Johnson High School    76.691117  77.491653  76.863248  77.187857
 Pena High School       83.372000  84.328125  84.121547  83.625455
 Rodriguez High School  76.612500  76.395626  77.690748  76.859966
 Shelton High School    82.917411  83.383495  83.778976  83.420755
 Thomas High School     83.087886  83.498795  83.497041  83.59

In [139]:
school_summary = school_data_complete.groupby('school_name').agg(
    School_Type=('type', 'first'),
    Total_Students=('Student ID', 'count'),
    Total_Budget=('budget', 'first'),
    Average_Math_Score=('math_score', 'mean'),
    Average_Reading_Score=('reading_score', 'mean')
).reset_index()

school_summary['Per Student Budget'] = school_summary['Total_Budget'] / school_summary['Total_Students']

passing_math_count = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name').size()
passing_reading_count = school_data_complete[school_data_complete['reading_score'] >= 70].groupby('school_name').size()

school_summary['Passing_Math_Count'] = passing_math_count
school_summary['Passing_Reading_Count'] = passing_reading_count

school_summary['Percentage Passing Math'] = (school_summary['Passing_Math_Count'] / school_summary['Total_Students']) * 100
school_summary['Percentage Passing Reading'] = (school_summary['Passing_Reading_Count'] / school_summary['Total_Students']) * 100

school_summary['Overall Passing Rate'] = (school_summary['Percentage Passing Math'] + school_summary['Percentage Passing Reading']) / 2

spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
school_summary['Spending Range'] = pd.cut(school_summary['Per Student Budget'], spending_bins, labels=group_names)

spending_summary = school_summary.groupby('Spending Range').agg(
    Average_Math_Score=('Average_Math_Score', 'mean'),
    Average_Reading_Score=('Average_Reading_Score', 'mean'),
    Passing_Math_Percentage=('Percentage Passing Math', 'mean'),
    Passing_Reading_Percentage=('Percentage Passing Reading', 'mean'),
    Overall_Passing_Rate=('Overall Passing Rate', 'mean')
).reset_index()

spending_summary


,Spending Range,Average_Math_Score,Average_Reading_Score,Passing_Math_Percentage,Passing_Reading_Percentage,Overall_Passing_Rate
0,<$585,83.455399,83.933814,NaN,NaN,NaN
1,$585-615,83.599686,83.885211,NaN,NaN,NaN
2,$615-645,79.079225,81.891436,NaN,NaN,NaN
3,$645-675,76.997210,81.027843,NaN,NaN,NaN


In [141]:

size_bins = [0, 1000, 2000, 5000]
size_group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


school_summary['School Size'] = pd.cut(school_summary['Total_Students'], size_bins, labels=size_group_names)

size_summary = school_summary.groupby('School Size').agg(
    Average_Math_Score=('Average_Math_Score', 'mean'),
    Average_Reading_Score=('Average_Reading_Score', 'mean'),
    Passing_Math_Percentage=('Percentage Passing Math', 'mean'),
    Passing_Reading_Percentage=('Percentage Passing Reading', 'mean'),
    Overall_Passing_Rate=('Overall Passing Rate', 'mean')
).reset_index()


size_summary


,School Size,Average_Math_Score,Average_Reading_Score,Passing_Math_Percentage,Passing_Reading_Percentage,Overall_Passing_Rate
0,Small (<1000),83.821598,83.929843,NaN,NaN,NaN
1,Medium (1000-2000),83.374684,83.864438,NaN,NaN,NaN
2,Large (2000-5000),77.746417,81.344493,NaN,NaN,NaN
